<a href="https://colab.research.google.com/github/kyochanpy/Kaggle_Indoor_Location_Navigation/blob/main/create_dataset/create_all_data_set_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 
import shutil
from sklearn.preprocessing import LabelEncoder

In [3]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [14]:
floor_accurate = pd.read_csv('/content/drive/MyDrive/submission_floor_accurate.csv').drop(['x', 'y'], axis=1)

In [26]:
train = pd.read_csv('/content/drive/MyDrive/all_data_floor/5a0546857ecc773753327266_train.csv')
test = pd.read_csv('/content/drive/MyDrive/all_data_floor/5a0546857ecc773753327266_test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3402,3403,3404,3408) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
display(train.head())
display(test.head())

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,...,accelerometer_x,accelerometer_y,accelerometer_z,accelerometer_uncalibrated_timestamp,accelerometer_uncalibrated_x,accelerometer_uncalibrated_y,accelerometer_uncalibrated_z,accelerometer_uncalibrated_x2,accelerometer_uncalibrated_y2,accelerometer_uncalibrated_z2,gyroscope_timestamp,gyroscope_x,gyroscope_y,gyroscope_z,gyroscope_uncalibrated_timestamp,gyroscope_uncalibrated_x,gyroscope_uncalibrated_y,gyroscope_uncalibrated_z,gyroscope_uncalibrated_x2,gyroscope_uncalibrated_y2,gyroscope_uncalibrated_z2,magnetic_field_timestamp,magnetic_field_x,magnetic_field_y,magnetic_field_z,magnetic_field_uncalibrated_timestamp,magnetic_field_uncalibrated_x,magnetic_field_uncalibrated_y,magnetic_field_uncalibrated_z,magnetic_field_uncalibrated_x2,magnetic_field_uncalibrated_y2,magnetic_field_uncalibrated_z2,rotation_vector_timestamp,rotation_vector_x,rotation_vector_y,rotation_vector_z,f,x,y,timestamp
0,0,0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,78.21819,38.999737,1561368762838
1,1,1,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,78.21819,38.999737,1561368762838
2,2,2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-0.349838,1.654572,10.940567,1.561369e+12,-0.679031,1.893387,11.749207,NaN,NaN,NaN,1.561369e+12,-0.101288,-0.296143,-0.092148,1.561369e+12,-0.099762,-0.296005,-0.091324,NaN,NaN,NaN,1.561369e+12,8.889771,-41.38031,-21.443176,1.561369e+12,-102.656555,-130.71594,-417.3462,NaN,NaN,NaN,1.561369e+12,-0.02873,0.134764,0.984518,2,76.25184,30.510840,1561368774057
3,3,3,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-0.349838,1.654572,10.940567,1.561369e+

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,...,beacon_timestamp2,accelerometer_timestamp,accelerometer_x,accelerometer_y,accelerometer_z,accelerometer_uncalibrated_timestamp,accelerometer_uncalibrated_x,accelerometer_uncalibrated_y,accelerometer_uncalibrated_z,accelerometer_uncalibrated_x2,accelerometer_uncalibrated_y2,accelerometer_uncalibrated_z2,gyroscope_timestamp,gyroscope_x,gyroscope_y,gyroscope_z,gyroscope_uncalibrated_timestamp,gyroscope_uncalibrated_x,gyroscope_uncalibrated_y,gyroscope_uncalibrated_z,gyroscope_uncalibrated_x2,gyroscope_uncalibrated_y2,gyroscope_uncalibrated_z2,magnetic_field_timestamp,magnetic_field_x,magnetic_field_y,magnetic_field_z,magnetic_field_uncalibrated_timestamp,magnetic_field_uncalibrated_x,magnetic_field_uncalibrated_y,magnetic_field_uncalibrated_z,magnetic_field_uncalibrated_x2,magnetic_field_uncalibrated_y2,magnetic_field_uncalibrated_z2,rotation_vector_timestamp,rotation_vector_x,rotation_vector_y,rotation_vector_z,floor,timestamp
0,0,0,4,-79,-999,-999,-76,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,-999,-52,-82,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
1,1,1,4,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-78,-999,-999,-999,-999,-76,-85,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,7
2,2,2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-83,-999,-999,-999,-999,-999,-81,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,7
3,3,3,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-89,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,7
4,4,4,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,

In [24]:
test['site_path_timestamp']

0      5a0546857ecc773753327266_ed77f28aeaf89b317bc38...
1      5a0546857ecc773753327266_dd4cbd69218f610f27cf3...
2      5a0546857ecc773753327266_bd0f2c8626679895f3338...
3      5a0546857ecc773753327266_ffcd9524c80c0fa5bb859...
4      5a0546857ecc773753327266_bd5921cd1008382e2f537...
                             ...                        
294    5a0546857ecc773753327266_986924433ab01afa81a59...
295    5a0546857ecc773753327266_6d89334316127640cff99...
296    5a0546857ecc773753327266_0c06cc9f21d172618d74c...
297    5a0546857ecc773753327266_986924433ab01afa81a59...
298    5a0546857ecc773753327266_dd4cbd69218f610f27cf3...
Name: site_path_timestamp, Length: 299, dtype: object

In [20]:
train_ = pd.read_csv('/content/drive/MyDrive/wifi_0/train/5a0546857ecc773753327266_train.csv')
test_ = pd.read_csv('/content/drive/MyDrive/wifi_0/test/5a0546857ecc773753327266_test.csv')
display(train_.head())
display(test_.head())

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd76579680a8142a19e69dcd6c33db61dc0a2444,fd7e06b88954f0ca0a2987e1741453624462bcc6,fd83a61cb82476f0ad6af28c4cd5872f285fd104,fd963f68e158c7e77bdd1d0a867927a1d270e972,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,timestamp,x,y,f,path
0,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-92,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-51,-999,-999,-999,-999,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,1578466886458,41.316772,180.01710,-1,5e1580bb1506f2000638fc62
1,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-92,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-54,-999,-999,-999,-999,-85,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,1578466893648,47.679752,178.77545,-1,5e1580bb1506f2000638fc62
2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-99

,Unnamed: 0,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,02eb2d69ac77b3bf6790139cb1a3b3c4571ecad4,02f764b7b09485106263e56ed9573966b48c0ad7,...,fd49231cf4345ad6354cad9942cf83ad3261b35b,fd5875bae44a87665f646e6c6e14c525b8950f6b,fd5da29ea31fc8fa2315f92f0c067e4e24e42b05,fd76579680a8142a19e69dcd6c33db61dc0a2444,fd7e06b88954f0ca0a2987e1741453624462bcc6,fd83a61cb82476f0ad6af28c4cd5872f285fd104,fd963f68e158c7e77bdd1d0a867927a1d270e972,fd977a3af7be241a9ed0213acb3aa75e5dc00253,fdb1ad87bd6fb08014267f2586faeed0edc7412b,fdc09ce75c596224ceb7e160e17c76f88453c246,fdc189e5a19850397f37201f4acc378cfddcf0d6,fdc19f011587b75c11a6c30d8ca06d90107b6bde,fdcce999b70500e2117e22cb9e1e5977ee28fbb3,fdf37fa13679f581bdfaae3b99e368633e0a144b,fdfad99711e053cd7dca1057f8dfb80b2f275728,fdfe926caf5f49a88a9bcab8d025e887f422128b,fe308dfc915f33c4217763c942ece65760cdf259,fe3211f90e4ab1f500e10fe175ae6142f4b13130,fe3461438b7a21c85a42ffd76030ece52e11dd7d,fe3c891bfb4d92361744aa697be30809c182020a,fe45b1591886927c2e9165618b4850019fd2606f,fe755bd56d9ed25f77b0d180a02c14e901af6332,fe994d2d43075b58afeee240f59149d3bed4660f,fe99e94b9d7235df4c50ec1b0ea9365e3ccdd5d3,feb15bdc09e48920ebf0b57460d7e70fe45c0553,feb165b7efdf8fdccb09368c512418917122f485,feb21e7aa4d21fba283d038b5cbf2ca33d929986,fec3a97cf01c9dfb1178693cc2924aabfbd4e1ef,ff079ef1efd3ec97d9914bc40e45aece8f680704,ff102b88d3012a2c798adee007c4a20172b0ffae,ff115962b9756008e3bd8b6a969d53b1ac7a4616,ff5c19eefe8884f125c3a91736fc0fb32ecdd919,ff70d33df144bdc870f2fd804d713e3c430f1c9b,ffa034716a62f0313b4cc2a05576e594b5ae7bdc,ffa41c79865d7fb336f586e0dec8b080db1027fb,ffc7c34369257431c7de9129094deb923bb3e3af,ffe53bd4dcfaa42668baf5ea0d2ddc676538fce0,fffedf27bf3bf8fa64d01e6b0a6affc9170716f1,timestamp,site_path_timestamp
0,4,-51,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-72,-999,-999,-999,-999,-999,-999,-999,-71,-55,-999,-75,-999,-999,-999,-999,-66,-47,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-83,-999,-999,-83,-999,-78,-999,-63,-999,-999,-61,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-999,-999,-999,-999,9,5a0546857ecc773753327266_046cfa46be49fc1083481...
1,4,-55,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-74,-65,-999,-82,-999,-999,-999,-999,-66,-65,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-77,-999,-999,-86,-999,-75,-999,-70,-999,-999,-57,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-71,-999,-89,

In [8]:
train['f']

0       2
1       2
2       2
3       2
4       2
       ..
9291   -1
9292   -1
9293   -1
9294   -1
9295   -1
Name: f, Length: 9296, dtype: int64

In [13]:
for building in used_buildings:
    df = pd.read_csv(f'/content/drive/MyDrive/all_data_24/train/{building}_train.csv')
    df_a = df.loc[:, ['f', 'x', 'y', 'timestamp']]
    df = df.drop(['f', 'x', 'y', 'timestamp'], axis=1)
    df['f'] = df_a['f'].values
    df['x'] = df_a['x'].values
    df['y'] = df_a['y'].values
    df['timestamp'] = df_a['timestamp'].values
    df.to_csv(f'{building}_train.csv')
    shutil.move(f'{building}_train.csv', '/content/drive/MyDrive/all_data_floor')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3405,3406,3407,3411) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3071,3072,3073,3077) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7037,7038,7039,7043) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4972,4973,4974,4978) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interacti

In [25]:
for building in used_buildings:
    df = pd.read_csv(f'/content/drive/MyDrive/all_data_24/test/{building}_test.csv')
    df_b = pd.merge(df, floor_accurate, on='site_path_timestamp')
    df['floor'] = df_b['floor'].values
    df_a = df.loc[:,['site_path_timestamp','timestamp']]
    df = df.drop('timestamp', axis=1)
    df['timestamp'], df['site_path_timestamp'] = df_a['timestamp'].values, df_a['site_path_timestamp'].values
    df.to_csv(f'{building}_test.csv')
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/all_data_floor')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1918,1919,1920,1924) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1314,1315,1316,1320) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3504,3505,3506,3510) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4524,4525,4526,4530) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interacti

In [ ]:
    shutil.move(f'{building}_test.csv', '/content/drive/MyDrive/all_data_floor')

In [16]:
c = pd.read_csv('/content/5a0546857ecc773753327266_test.csv')
c

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,000840e5c600de293cea57f13326f273c86c3988,0023c80b0e4cb8338cf25de26143bb0570a16471,005246b6f51feb1a069e8f005d3e6aba2591b65b,0076ff7a084cb2ac8c146139965ab1be296e72c4,0089ad1dd75b13e2c3ceda344988c9f89a83a2f9,009a3ed672be7bd1b9c4437b43a53296771af098,00ad587dcb9c7ce3788b92e22777a22ee0efea31,00af060fc145ee6a6a50475efa57b91cbf54237f,00bcc61bdea4d52d050822d66952dd707c2fcdf3,00c8933965e23cd2bb890ca08ee7f8a22f933df8,00e2787f25a1d770f62ed57d08b9393c7105b00d,00ef32991244ca6ce16c9ee3b910886401766bf6,00f0904087c01d922d6ebf3005607dfdeaf6687b,00f09c4561768a871af2921cffa0a4fda515766b,00f36a2e037357f98009f8d084fa70eb5071aa42,0102fdd6f12bee16a21bb62cd67afa00d2272670,011aac59a4af0f887f3914bf0dca6263625e9c5b,011e20ebf721a1c6dfec42e8ed1e2ac566073a2a,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,015504eea807d723f1d1a1aca43544189830aed7,01846fc4449a414413024fbc6d2e9be1a2203f3c,0192ce7fc9f7e169d3edd58151d50555d70ebb27,019f793ca026ac0c1489ba3a199984aa7322ac55,01a39206e3987e6708631b62b70ecfd55ab05a7d,01c325eeb378ebe2ab3212fe86bf0060f71ced22,01d2f676abab6ec03ec5dc696bfd49d66e392ea1,01da12eb27c6e37d03d691c31b40591a724db4a2,01dd00cd784a639f7d6609ddc8bfc6c9af3b47d3,01e25e4a25acd32baf5137b3031151f751fadbb4,01ead5ee97d0329b2c228fece5a96b0358a3cab1,0231e64a6307175a78f12cfe001c579e580ed721,0235185b1ffbe48499aa88fad9f58aad7c31832f,026c2f057932da75680b21ecdbd23bf9cb9350f3,028a310e23177c3747d37971678dd964ee28ce17,02a1be3a5dab38320f879489d8a1e0f2a72768b3,02d90a0bc95e068a8f75e00b3bd78579cb996c89,02d93bd28ad16b985328fa798b164c54bd8dcc0c,...,beacon_timestamp2,accelerometer_timestamp,accelerometer_x,accelerometer_y,accelerometer_z,accelerometer_uncalibrated_timestamp,accelerometer_uncalibrated_x,accelerometer_uncalibrated_y,accelerometer_uncalibrated_z,accelerometer_uncalibrated_x2,accelerometer_uncalibrated_y2,accelerometer_uncalibrated_z2,gyroscope_timestamp,gyroscope_x,gyroscope_y,gyroscope_z,gyroscope_uncalibrated_timestamp,gyroscope_uncalibrated_x,gyroscope_uncalibrated_y,gyroscope_uncalibrated_z,gyroscope_uncalibrated_x2,gyroscope_uncalibrated_y2,gyroscope_uncalibrated_z2,magnetic_field_timestamp,magnetic_field_x,magnetic_field_y,magnetic_field_z,magnetic_field_uncalibrated_timestamp,magnetic_field_uncalibrated_x,magnetic_field_uncalibrated_y,magnetic_field_uncalibrated_z,magnetic_field_uncalibrated_x2,magnetic_field_uncalibrated_y2,magnetic_field_uncalibrated_z2,rotation_vector_timestamp,rotation_vector_x,rotation_vector_y,rotation_vector_z,timestamp,floor
0,0,0,4,-79,-999,-999,-76,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,-999,-52,-82,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
1,1,1,4,-91,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-78,-999,-999,-999,-999,-76,-85,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2
2,2,2,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-83,-999,-999,-999,-999,-999,-81,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,-1
3,3,3,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-89,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,-1
4,4,4,4,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,